In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import librosa
#import gzip


##with gzip.open('cv-corpus-12.0-2022-12-07-as.tar.gz', 'rb') as f:
  #  file_content = f.read()
   # audio_data, sr = librosa.load(file_content)
	
import gzip
import librosa

#with gzip.open('cv-corpus-12.0-2022-12-07-as.tar.gz', 'rb') as f:
 #   file_content = f.read()
  #  audio_files = []
   # start = 0
    #while start < len(file_content):
     #   audio, sr = librosa.load(file_content[start:])
      #  audio_files.append((audio, sr))
       # start += len(audio)

import os
import librosa

##folder_path = r'C:\Users\chawl\Desktop\Untitled Folder\TRI-NIT\clip'
audio_files = []

#for filename in os.listdir(folder):
 #   if filename.endswith('.wav') or filename.endswith('.mp3') or filename.endswith('.flac'):
  #      audio_path = os.path.join(folder, filename)
   #     audio, sr = librosa.load(audio_path)
    #    audio_files.append((audio, sr))
        


##for filename in os.listdir(folder_path):
  ##  file_path = os.path.join(folder_path, filename)
    ##audio, sr = librosa.load(file_path)
    ##if len(audio) == 0:
      ##  print("Error: Input audio signal is empty.")
    ##else:
      ##  resampled_audio = librosa.resample(audio, sr, 22050)
        # Do something with the resampled audio signal
        ##audio_files.append((resampled_audio, sr))

# Load the audio dataset
#audio_files = [librosa.load(file) for file in res[:5]]
#audio_files = res[:5]


audio , sr = librosa.load(r"C:\Users\chawl\Desktop\Untitled Folder\TRI-NIT\clip\1.wav")
audio_files.append((audio, sr))




import numpy as np

def preprocess(file):
    audio, sr = file
    audio = np.array(audio, dtype=np.float32)
    return audio

x_train = [preprocess(file) for file in audio_files]
#x_train = [preprocess(file) for file, sr in audio_files]
#x_train = np.array(x_train)

# Define the generator model
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = nn.Linear(100, 128)
        self.conv1 = nn.Conv2d(128, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 1, 3, padding=1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = x.view(-1, 128, 1, 1)
        x = F.interpolate(x, size=(128, 128), mode='nearest')
        x = F.relu(self.conv1(x))
        x = torch.sigmoid(self.conv2(x))
        return x

# Define the discriminator model
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, 3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 16 * 16, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 128 * 16 * 16)
        x = torch.sigmoid(self.fc1(x))
        return x

# Train the GAN
generator = Generator()
discriminator = Discriminator()
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002)
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002)

for epoch in range(200):
    for i, (audio, _) in enumerate(x_train):
        # Train the discriminator
        discriminator.zero_grad()
        real_labels = torch.ones(1, 1)
        real_output = discriminator(audio)
        real_loss = F.binary_cross_entropy(real_output, real_labels)
        real_loss.backward()

        noise = torch.randn(1, 100)
        fake_audio = generator(noise)
        fake_labels = torch.zeros(1, 1)
        fake_output = discriminator(fake_audio)
        fake_loss = F.binary_cross_entropy(fake_output, fake_labels)
        fake_loss.backward()

        d_optimizer.step()

        # Train the generator
        generator.zero_grad()
        noise = torch.randn(1, 100)
        fake_audio = generator(noise)
        output = discriminator(fake_audio)
        g_loss = F.binary_cross_entropy(output, real_labels)
        g_loss.backward()

        g_optimizer.step()

        if i % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}'
                  .format(epoch, 200, i, len(x_train), real_loss + fake_loss, g_loss))


# Generate new audio
noise = torch.randn(1, 100)
generated_audio = generator(noise)
generated_audio = np.squeeze(generated_audio.detach().numpy())
generated_audio = generated_audio * (data.max() - data.min()) + data.min()
librosa.display.specshow(librosa.power_to_db(generated_audio, ref=np.max), sr=sr, x_axis='time', y_axis='mel')